<a href="https://colab.research.google.com/github/yasahiro10/ML_projects/blob/main/Cancer_Classificaiton_using_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>